In [ ]:
import numpy as np
import pandas as pd

from fastai import *
from fastai.vision import *

import imageio

import os

import matplotlib.pyplot as plt
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## Control Variables

In [ ]:
root = Path('../input')
train_path = Path('train')
rseed = 7
val_size = 0.05

# Helper functions

In [ ]:
def save_imgs(path:Path, data, labels):
    path.mkdir(parents=True,exist_ok=True)
    for label in np.unique(labels):
        (path/str(label)).mkdir(parents=True,exist_ok=True)
    for i in range(len(data)):
        if(len(labels)!=0):
            imageio.imsave( str( path/str(labels[i])/(str(i)+'.jpg') ), data[i] )
        else:
            imageio.imsave( str( path/(str(i)+'.jpg') ), data[i] )

# Load data

In [ ]:
train_csv = pd.read_csv(root/'train.csv')

In [ ]:
test_csv = pd.read_csv(root/'test.csv')

# Explore data

In [ ]:
train_csv.head()

# Prepare data

In [ ]:
data_X, data_y = train_csv.loc[:,'pixel0':'pixel783'], train_csv['label']

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(data_X, data_y, test_size=val_size,random_state=rseed,stratify=data_y)

In [ ]:
def to_img_shape(data_X, data_y=[]):
    data_X = np.array(data_X).reshape(-1,28,28)
    data_X = np.stack((data_X,)*3, axis=-1)
    data_y = np.array(data_y)
    return data_X,data_y

In [ ]:
train_X,train_y = to_img_shape(train_X, train_y)

In [ ]:
val_X,val_y = to_img_shape(val_X,val_y)

Save images

In [ ]:
save_imgs(Path('/data/train'),train_X,train_y)

In [ ]:
save_imgs(Path('/data/valid'),val_X,val_y)

In [ ]:
data = ImageDataBunch.from_folder('/data/',bs=256,size=28,ds_tfms=get_transforms(do_flip=False),num_workers=0).normalize(imagenet_stats)

In [ ]:
data.show_batch(3,figsize=(6,6))

# Train model

In [ ]:
learn = cnn_learner(data,models.resnet18,metrics=accuracy,path='.')

In [ ]:
learn.fit_one_cycle(4,0.01)

In [ ]:
learn.save('s1')

# Fine tune

In [ ]:
learn.load('s1');

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(30,1e-4)

# Interpret results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(8,8))

In [ ]:
interp.plot_top_losses(9,figsize=(9,9))

# Make predictions

In [ ]:
sub_df = pd.DataFrame(columns=['ImageId','Label'])

In [ ]:
test_data = np.array(test_csv)

In [ ]:
def get_img(data):
    t1 = data.reshape(28,28)/255
    t1 = np.stack([t1]*3,axis=0)
    img = Image(FloatTensor(t1))
    return img

In [ ]:
from fastprogress import progress_bar

In [ ]:
mb=progress_bar(range(test_data.shape[0]))
for i in mb:
    timg=test_data[i]
    img = get_img(timg)
    sub_df.loc[i]=[i+1,int(learn.predict(img)[1])]

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv',index=False)